## PySAL Change Log Statistics: Table Generation

This notebook generates the summary statistics for use in the 6-month releases of PySAL, which is now a meta package. 

It assumes the subpackages have been git cloned in a directory below the location of this notebook. It also requires network connectivity for some of the reporting.

Run this notebook after `100-gitcount.ipynb`


In [1]:
from __future__ import print_function
import os
import json
import re
import sys
import pandas
import subprocess###
from subprocess import check_output

#import yaml
from datetime import datetime, timedelta, time

from dateutil.parser import parse
import pytz

utc=pytz.UTC

try:
    from urllib import urlopen
except:
    from urllib.request import urlopen

from release_info import release_date, start_date, PYSALVER

since = datetime.combine(start_date, time(0,0))


/tmp/ipykernel_279048/3190583148.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
release_date

datetime.date(2024, 7, 14)

In [3]:
CWD = os.path.abspath(os.path.curdir)

In [4]:
CWD

'/home/serge/para/1_projects/code-pysal-meta/pysal/tools'

In [5]:
with open('frozen.txt', 'r') as package_list:
    packages = package_list.readlines()
    packages = dict([package.strip().split(">=") for package in packages])

In [6]:
import release_info

In [7]:
release_info.PYSALVER

'24.07rc2'

In [8]:
#import pysal
packages['pysal'] = release_info.PYSALVER

In [9]:
packages

{'libpysal': '4.11.0',
 'access': '1.1.9',
 'esda': '2.5.1',
 'giddy': '2.3.5',
 'inequality': '1.0.1',
 'pointpats': '2.5.0',
 'segregation': '2.5',
 'spaghetti': '1.7.6',
 'mgwr': '2.2.1',
 'momepy': '0.7.2',
 'spglm': '1.1.0',
 'spint': '1.0.7',
 'spreg': '1.5.0',
 'spvcm': '0.3.0',
 'tobler': '0.11.2',
 'mapclassify': '2.7.0',
 'splot': '1.1.5.post1',
 'spopt': '0.6.1',
 'pysal': '24.07rc2'}

In [10]:
import pickle

In [11]:
issues_closed = pickle.load(open("issues_closed.p", 'rb'))
pulls_closed = pickle.load(open('pulls_closed.p', 'rb'))

In [12]:
type(issues_closed)

dict

In [13]:
issues_closed.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [14]:
from release_info import get_pypi_info, get_github_info, clone_masters

In [15]:
#github_releases = get_github_info()

github_releases = pickle.load(open("releases.p", 'rb'))


In [16]:
from datetime import datetime

In [17]:
#pysal_date = datetime.strptime('2021-07-31T12:00:00Z', '%Y-%m-%dT%H:%M:%SZ')
pysal_date = release_date
#ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


In [18]:
from datetime import date, datetime, time

# Create a date object
date_obj = release_date

# Create a time object (optional, default is midnight if not specified)
time_obj = time(0, 0)

# Combine date and time to create a datetime object
datetime_obj = datetime.combine(date_obj, time_obj)


pysal_rel = {'version': f'v{PYSALVER}',
            'release_date': datetime_obj}
github_releases['pysal'] = pysal_rel

In [19]:
github_releases

{'libpysal': {'version': 'v4.11.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.11.0',
  'release_date': datetime.datetime(2024, 6, 10, 19, 39, 46)},
 'access': {'version': 'v1.1.9',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.9',
  'release_date': datetime.datetime(2023, 10, 6, 2, 43, 55)},
 'esda': {'version': 'v2.5.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.5.1',
  'release_date': datetime.datetime(2023, 10, 24, 17, 26, 14)},
 'giddy': {'version': 'v2.3.5',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.5',
  'release_date': datetime.datetime(2024, 1, 16, 22, 39, 24)},
 'inequality': {'version': 'v1.0.1',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.1',
  'release_date': datetime.datetime(2023, 10, 28, 23, 10, 33)},
 'pointpats': {'version': 'v2.5.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.5.0',
  'release_date': datetime.datetime(2024, 7, 4, 5, 5, 17)}

In [20]:
from datetime import datetime
datetime.fromtimestamp(0)
ISO8601 = "%Y-%m-%dT%H:%M:%SZ"


final_pulls = {}
final_issues = {}
for package in packages:
    filtered_issues = []
    filtered_pulls = []
    released = github_releases[package]['release_date']
    print(f'package {package} released {released} {type(released)}')
    package_pulls = pulls_closed[package]
    package_issues = issues_closed[package]
    for issue in package_issues:
        #print(issue['number'], issue['title'], issue['closed_at'])
        closed = datetime.strptime(issue['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_issues.append(issue)
    final_issues[package] = filtered_issues
    for pull in package_pulls:
        #print(pull['number'], pull['title'], pull['closed_at'])
        closed = datetime.strptime(pull['closed_at'], ISO8601)
        if closed <= released and closed > since:
            filtered_pulls.append(pull)
    final_pulls[package] = filtered_pulls
    print(package, released, len(package_issues), len(filtered_issues), len(package_pulls),
         len(filtered_pulls))

package libpysal released 2024-06-10 19:39:46 <class 'datetime.datetime'>
libpysal 2024-06-10 19:39:46 64 36 0 0
package access released 2023-10-06 02:43:55 <class 'datetime.datetime'>
access 2023-10-06 02:43:55 1 0 0 0
package esda released 2023-10-24 17:26:14 <class 'datetime.datetime'>
esda 2023-10-24 17:26:14 36 0 0 0
package giddy released 2024-01-16 22:39:24 <class 'datetime.datetime'>
giddy 2024-01-16 22:39:24 17 0 0 0
package inequality released 2023-10-28 23:10:33 <class 'datetime.datetime'>
inequality 2023-10-28 23:10:33 4 0 3 0
package pointpats released 2024-07-04 05:05:17 <class 'datetime.datetime'>
pointpats 2024-07-04 05:05:17 9 8 6 5
package segregation released 2023-08-24 16:41:39 <class 'datetime.datetime'>
segregation 2023-08-24 16:41:39 3 0 0 0
package spaghetti released 2024-06-21 13:02:05 <class 'datetime.datetime'>
spaghetti 2024-06-21 13:02:05 14 10 0 0
package mgwr released 2024-01-06 16:43:47 <class 'datetime.datetime'>
mgwr 2024-01-06 16:43:47 4 0 0 0
package

In [21]:
type(issue['closed_at'])

str

In [22]:
package

'pysal'

In [23]:
issue

{'url': 'https://api.github.com/repos/pysal/pysal/issues/1297',
 'repository_url': 'https://api.github.com/repos/pysal/pysal',
 'labels_url': 'https://api.github.com/repos/pysal/pysal/issues/1297/labels{/name}',
 'comments_url': 'https://api.github.com/repos/pysal/pysal/issues/1297/comments',
 'events_url': 'https://api.github.com/repos/pysal/pysal/issues/1297/events',
 'html_url': 'https://github.com/pysal/pysal/pull/1297',
 'id': 2103915575,
 'node_id': 'PR_kwDOAH6T1M5lPlVp',
 'number': 1297,
 'title': 'update chat links: gitter >> discord',
 'user': {'login': 'jGaboardi',
  'id': 8590583,
  'node_id': 'MDQ6VXNlcjg1OTA1ODM=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/8590583?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/jGaboardi',
  'html_url': 'https://github.com/jGaboardi',
  'followers_url': 'https://api.github.com/users/jGaboardi/followers',
  'following_url': 'https://api.github.com/users/jGaboardi/following{/other_user}',
  'gists_url': 'http

In [24]:
issue_details = final_issues
pull_details = final_pulls

In [25]:
packages

{'libpysal': '4.11.0',
 'access': '1.1.9',
 'esda': '2.5.1',
 'giddy': '2.3.5',
 'inequality': '1.0.1',
 'pointpats': '2.5.0',
 'segregation': '2.5',
 'spaghetti': '1.7.6',
 'mgwr': '2.2.1',
 'momepy': '0.7.2',
 'spglm': '1.1.0',
 'spint': '1.0.7',
 'spreg': '1.5.0',
 'spvcm': '0.3.0',
 'tobler': '0.11.2',
 'mapclassify': '2.7.0',
 'splot': '1.1.5.post1',
 'spopt': '0.6.1',
 'pysal': '24.07rc2'}

In [26]:
github_releases['pysal']['release_date'] = release_date

In [27]:
released

datetime.datetime(2024, 7, 14, 0, 0)

In [28]:
packages.keys()

dict_keys(['libpysal', 'access', 'esda', 'giddy', 'inequality', 'pointpats', 'segregation', 'spaghetti', 'mgwr', 'momepy', 'spglm', 'spint', 'spreg', 'spvcm', 'tobler', 'mapclassify', 'splot', 'spopt', 'pysal'])

In [29]:
spvcm = packages['spvcm']

In [30]:
## skip packages not released since last meta release

# handle meta
mrd = github_releases['pysal']['release_date']
github_releases['pysal']['release_date'] =  datetime.combine(mrd, time(0,0))


In [31]:
for package in github_releases:
    if github_releases[package]['release_date']>since:
        print("new: ",package)
    else:
        print('old:', package)

new:  libpysal
old: access
old: esda
old: giddy
old: inequality
new:  pointpats
old: segregation
new:  spaghetti
old: mgwr
new:  momepy
old: spglm
old: spint
new:  spreg
old: spvcm
old: tobler
new:  mapclassify
old: splot
new:  spopt
new:  pysal


In [32]:
github_releases[package]['release_date']

datetime.datetime(2024, 7, 14, 0, 0)

In [33]:
since

datetime.datetime(2024, 1, 31, 0, 0)

In [34]:
github_releases

{'libpysal': {'version': 'v4.11.0',
  'url': 'https://api.github.com/repos/pysal/libpysal/tarball/v4.11.0',
  'release_date': datetime.datetime(2024, 6, 10, 19, 39, 46)},
 'access': {'version': 'v1.1.9',
  'url': 'https://api.github.com/repos/pysal/access/tarball/v1.1.9',
  'release_date': datetime.datetime(2023, 10, 6, 2, 43, 55)},
 'esda': {'version': 'v2.5.1',
  'url': 'https://api.github.com/repos/pysal/esda/tarball/v2.5.1',
  'release_date': datetime.datetime(2023, 10, 24, 17, 26, 14)},
 'giddy': {'version': 'v2.3.5',
  'url': 'https://api.github.com/repos/pysal/giddy/tarball/v2.3.5',
  'release_date': datetime.datetime(2024, 1, 16, 22, 39, 24)},
 'inequality': {'version': 'v1.0.1',
  'url': 'https://api.github.com/repos/pysal/inequality/tarball/v1.0.1',
  'release_date': datetime.datetime(2023, 10, 28, 23, 10, 33)},
 'pointpats': {'version': 'v2.5.0',
  'url': 'https://api.github.com/repos/pysal/pointpats/tarball/v2.5.0',
  'release_date': datetime.datetime(2024, 7, 4, 5, 5, 17)}

In [35]:
since_date = '--since="{start}"'.format(start=start_date.strftime("%Y-%m-%d"))
since_date
                                        

'--since="2024-01-31"'

In [36]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
tag_dates = {}
ncommits_total = 0
for subpackage in packages:
    released = github_releases[subpackage]['release_date']
    tag_date = released.strftime("%Y-%m-%d")
    tag_dates[subpackage] = tag_date
    print(tag_date)
    #tag_date = tag_dates[subpackage]
    ncommits = 0
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        ncommits_total = len(check_output(cmd).splitlines())
    print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

2024-06-10
libpysal 78 41 2024-06-10
2023-10-06
access 78 0 2023-10-06
2023-10-24
esda 78 0 2023-10-24
2024-01-16
giddy 78 0 2024-01-16
2023-10-28
inequality 78 0 2023-10-28
2024-07-04
pointpats 18 18 2024-07-04
2023-08-24
segregation 18 0 2023-08-24
2024-06-21
spaghetti 15 11 2024-06-21
2024-01-06
mgwr 15 0 2024-01-06
2024-06-27
momepy 74 69 2024-06-27
2023-10-25
spglm 74 0 2023-10-25
2020-09-09
spint 74 0 2020-09-09
2024-07-03
spreg 29 20 2024-07-03
2020-02-02
spvcm 29 0 2020-02-02
2023-09-26
tobler 29 0 2023-09-26
2024-07-04
mapclassify 52 48 2024-07-04
2022-04-13
splot 52 0 2022-04-13
2024-06-20
spopt 22 18 2024-06-20
2024-07-14
pysal 66 66 2024-07-14


In [37]:
activity

{'libpysal': 41,
 'access': 0,
 'esda': 0,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 18,
 'segregation': 0,
 'spaghetti': 11,
 'mgwr': 0,
 'momepy': 69,
 'spglm': 0,
 'spint': 0,
 'spreg': 20,
 'spvcm': 0,
 'tobler': 0,
 'mapclassify': 48,
 'splot': 0,
 'spopt': 18,
 'pysal': 66}

In [38]:
subpackage

'pysal'

In [39]:
CWD

'/home/serge/para/1_projects/code-pysal-meta/pysal/tools'

In [40]:
# commits
cmd = ['git', 'log', '--oneline', since_date]

activity = {}
total_commits = 0
for subpackage in packages:
    ncommits = 0
    tag_date = tag_dates[subpackage]
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        ncommits = len(check_output(cmd_until).splitlines())
        print(ncommits)
        ncommits_total = len(check_output(cmd).splitlines())
        print(subpackage, ncommits_total, ncommits, tag_date)
    total_commits += ncommits
    activity[subpackage] = ncommits

41
libpysal 78 41 2024-06-10
18
pointpats 18 18 2024-07-04
11
spaghetti 15 11 2024-06-21
69
momepy 74 69 2024-06-27
20
spreg 29 20 2024-07-03
48
mapclassify 52 48 2024-07-04
18
spopt 22 18 2024-06-20
66
pysal 66 66 2024-07-14


In [41]:
identities = {'Levi John Wolf': ('ljwolf', 'Levi John Wolf'),
              'Serge Rey': ('Serge Rey', 'Sergio Rey', 'sjsrey', 'serge'),
              'Wei Kang': ('Wei Kang', 'weikang9009'),
              'Dani Arribas-Bel': ('Dani Arribas-Bel', 'darribas'),
              'Antti Härkönen': ( 'antth', 'Antti Härkönen', 'Antti Härkönen', 'Antth'  ),
              'Juan C Duque': ('Juan C Duque', "Juan Duque"),
              'Renan Xavier Cortes': ('Renan Xavier Cortes', 'renanxcortes', 'Renan Xavier Cortes'   ),
              'Taylor Oshan': ('Tayloroshan', 'Taylor Oshan', 'TaylorOshan'),
              'Tom Gertin': ('@Tomgertin', 'Tom Gertin', '@tomgertin')
}

def regularize_identity(string):
    string = string.decode()
    for name, aliases in identities.items():
        for alias in aliases:
            if alias in string:
                string = string.replace(alias, name)
    if len(string.split(' '))>1:
        string = string.title()
    return string.lstrip('* ')

In [42]:
author_cmd = ['git', 'log', '--format=* %aN', since_date]

In [43]:
author_cmd.append('blank')

In [44]:
author_cmd

['git', 'log', '--format=* %aN', '--since="2024-01-31"', 'blank']

In [45]:
from collections import Counter

In [46]:
packages

{'libpysal': '4.11.0',
 'access': '1.1.9',
 'esda': '2.5.1',
 'giddy': '2.3.5',
 'inequality': '1.0.1',
 'pointpats': '2.5.0',
 'segregation': '2.5',
 'spaghetti': '1.7.6',
 'mgwr': '2.2.1',
 'momepy': '0.7.2',
 'spglm': '1.1.0',
 'spint': '1.0.7',
 'spreg': '1.5.0',
 'spvcm': '0.3.0',
 'tobler': '0.11.2',
 'mapclassify': '2.7.0',
 'splot': '1.1.5.post1',
 'spopt': '0.6.1',
 'pysal': '24.07rc2'}

In [47]:
tag_dates

{'libpysal': '2024-06-10',
 'access': '2023-10-06',
 'esda': '2023-10-24',
 'giddy': '2024-01-16',
 'inequality': '2023-10-28',
 'pointpats': '2024-07-04',
 'segregation': '2023-08-24',
 'spaghetti': '2024-06-21',
 'mgwr': '2024-01-06',
 'momepy': '2024-06-27',
 'spglm': '2023-10-25',
 'spint': '2020-09-09',
 'spreg': '2024-07-03',
 'spvcm': '2020-02-02',
 'tobler': '2023-09-26',
 'mapclassify': '2024-07-04',
 'splot': '2022-04-13',
 'spopt': '2024-06-20',
 'pysal': '2024-07-14'}

In [48]:
authors_global = set()
authors = {}
global_counter = Counter()
counters = dict()
cmd = ['git', 'log', '--oneline', since_date]
total_commits = 0
activity = {}
for subpackage in packages:
    ncommits = 0
    released = github_releases[subpackage]['release_date']
    if released > since:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        print(cmd)
        tag_date = tag_dates[subpackage]
        tag_date = (datetime.strptime(tag_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        author_cmd[-1] = '--until="{tag_date}"'.format(tag_date=tag_date)
        #cmd_until = cmd + ['--until="{tag_date}"'.format(tag_date=tag_date)]
        print(subpackage, author_cmd)


        all_authors = check_output(author_cmd).splitlines()
        counter = Counter([regularize_identity(author) for author in all_authors])
        global_counter += counter
        counters.update({subpackage: counter})
        unique_authors = sorted(set(all_authors))
        authors[subpackage] =  unique_authors
        authors_global.update(unique_authors)
    total_commits += ncommits
    activity[subpackage] = ncommits

['git', 'log', '--oneline', '--since="2024-01-31"']
libpysal ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-06-11"']
['git', 'log', '--oneline', '--since="2024-01-31"']
pointpats ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-07-05"']
['git', 'log', '--oneline', '--since="2024-01-31"']
spaghetti ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-06-22"']
['git', 'log', '--oneline', '--since="2024-01-31"']
momepy ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-06-28"']
['git', 'log', '--oneline', '--since="2024-01-31"']
spreg ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-07-04"']
['git', 'log', '--oneline', '--since="2024-01-31"']
mapclassify ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-07-05"']
['git', 'log', '--oneline', '--since="2024-01-31"']
spopt ['git', 'log', '--format=* %aN', '--since="2024-01-31"', '--until="2024-06-21"']
['git', '

In [49]:
author_cmd

['git',
 'log',
 '--format=* %aN',
 '--since="2024-01-31"',
 '--until="2024-07-15"']

In [50]:
subpackage

'pysal'

In [51]:
authors_global

{b'* Anna Brazdova',
 b'* Daniela Dan\xc4\x8dejov\xc3\xa1',
 b'* Germano Barcelos',
 b'* James Gaboardi',
 b'* Krasen Samardzhiev',
 b'* Levi John Wolf',
 b'* Lisa',
 b'* Marek Novotn\xc3\xbd',
 b'* Martin Fleischmann',
 b'* Pedro Amaral',
 b'* Serge Rey',
 b'* Sergio Rey',
 b'* anastassiavybornova',
 b'* dependabot[bot]',
 b'* eli knaap',
 b'* pre-commit-ci[bot]',
 b'* ryanzomorrodi',
 b'* stevee404'}

In [52]:
activity

{'libpysal': 78,
 'access': 0,
 'esda': 0,
 'giddy': 0,
 'inequality': 0,
 'pointpats': 18,
 'segregation': 0,
 'spaghetti': 15,
 'mgwr': 0,
 'momepy': 74,
 'spglm': 0,
 'spint': 0,
 'spreg': 29,
 'spvcm': 0,
 'tobler': 0,
 'mapclassify': 52,
 'splot': 0,
 'spopt': 22,
 'pysal': 66}

In [53]:
def get_tag(title, level="##", as_string=True):
    words = title.split()
    tag = "-".join([word.lower() for word in words])
    heading = level+" "+title
    line = "\n\n<a name=\"{}\"></a>".format(tag)
    lines = [line]
    lines.append(heading)
    if as_string:
        return "\n".join(lines)
    else:
        return lines

In [54]:
subs = issue_details.keys()
table = []
txt = []
lines = get_tag("Changes by Package", as_string=False)

for sub in github_releases:
    total= issue_details[sub]
    pr = pull_details[sub]
    
    row = [sub, activity[sub], len(total), len(pr)]
    table.append(row)
    #line = "\n<a name=\"{sub}\"></a>".format(sub=sub)
    #lines.append(line)
    #line = "### {sub}".format(sub=sub)
    #lines.append(line)
    sub_lower = sub.lower()
    sub_version = github_releases[sub_lower]['version']
    print(f'{sub_lower}, {sub_version}')
    title = f'{sub_lower} {sub_version}'
    lines.extend(get_tag(title, "###", as_string=False))
    for issue in total:
        url = issue['html_url']
        title = issue['title']
        number = issue['number']
        line = "* [#{number}:]({url}) {title} ".format(title=title,
                                                     number=number,
                                                     url=url)
        lines.append(line)



libpysal, v4.11.0
access, v1.1.9
esda, v2.5.1
giddy, v2.3.5
inequality, v1.0.1
pointpats, v2.5.0
segregation, v2.5
spaghetti, v1.7.6
mgwr, v2.2.1
momepy, v0.7.2
spglm, v1.1.0
spint, v1.0.7
spreg, v1.5.0
spvcm, v0.3.0
tobler, v0.11.2
mapclassify, v2.7.0
splot, v1.1.5.post1
spopt, v0.6.1
pysal, v24.07rc2


In [55]:
sub

'pysal'

In [56]:
os.chdir(CWD)

import pandas

In [57]:
df = pandas.DataFrame(table, columns=['package', 'commits', 'total issues', 'pulls'])

In [58]:
df.head()

,package,commits,total issues,pulls
0,libpysal,78,36,0
1,access,0,0,0
2,esda,0,0,0
3,giddy,0,0,0
4,inequality,0,0,0


In [59]:
df.shape

(19, 4)

In [60]:
type(counters)

dict

In [61]:
df.sort_values(['commits','pulls'], ascending=False)\
  .to_html('./commit_table.html', index=None)

In [62]:
df.sum()

package         libpysalaccessesdagiddyinequalitypointpatssegr...
commits                                                       354
total issues                                                  212
pulls                                                          12
dtype: object

In [63]:
contributor_table = pandas.DataFrame.from_dict(counters).fillna(0).astype(int).T

In [64]:
contributor_table.to_html('./contributor_table.html')

In [65]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits')

,commits
Anastassiavybornova,1
Anna Brazdova,1
Daniela Dančejová,1
Dependabot[Bot],10
Eli Knaap,51
Germano Barcelos,5
James Gaboardi,46
Krasen Samardzhiev,21
Levi John Wolf,1
Lisa,1


In [66]:
totals = contributor_table.sum(axis=0).T
totals.sort_index().to_frame('commits').to_html('./commits_by_person.html')

In [67]:
totals

Martin Fleischmann     79
Serge Rey              49
Krasen Samardzhiev     21
James Gaboardi         46
Lisa                    1
Anna Brazdova           1
Marek Novotný           1
Daniela Dančejová       1
Pre-Commit-Ci[Bot]     11
Levi John Wolf          1
Dependabot[Bot]        10
Eli Knaap              51
Stevee404               1
Anastassiavybornova     1
Pedro Amaral           12
Germano Barcelos        5
Ryanzomorrodi           1
dtype: int64

In [68]:
n_commits = df.commits.sum()
n_issues = df['total issues'].sum()
n_pulls = df.pulls.sum()

In [69]:
n_commits

354

In [70]:
#Overall, there were 719 commits that closed 240 issues, together with 105 pull requests across 12 packages since our last release on 2017-11-03.
#('{0} Here is a really long '
#           'sentence with {1}').format(3, 5))
line = ('Overall, there were {n_commits} commits that closed {n_issues} issues'  
    ' since our last release' 
        ' on {since_date}.\n'.format(n_commits=n_commits, n_issues=n_issues,
        since_date = start_date))

In [71]:
line

'Overall, there were 354 commits that closed 212 issues since our last release on 2024-01-31.\n'

## append html files to end of changes.md with tags for toc

In [72]:
with open('changes.md', 'w') as fout:
    fout.write(line)
    fout.write("\n".join(lines))
    fout.write(get_tag("Contributors"))
    fout.write("\n\nMany thanks to all of the following individuals who contributed to this release:\n\n")
    
    
    
    totals = contributor_table.sum(axis=0).T
    contributors = totals.index.values
    contributors.sort()
    contributors = contributors.tolist() 
    contributors = [ f'\n - {contributor}' for contributor in contributors]
    fout.write("".join(contributors))
    


In [73]:
df.head()

,package,commits,total issues,pulls
0,libpysal,78,36,0
1,access,0,0,0
2,esda,0,0,0
3,giddy,0,0,0
4,inequality,0,0,0


In [74]:
df.head(17)

,package,commits,total issues,pulls
0,libpysal,78,36,0
1,access,0,0,0
2,esda,0,0,0
3,giddy,0,0,0
4,inequality,0,0,0
5,pointpats,18,8,5
6,segregation,0,0,0
7,spaghetti,15,10,0
8,mgwr,0,0,0
9,momepy,74,82,0


In [75]:
n_pulls

12

In [76]:
"""
Update ../pyproject.toml for minimum pysal package pinning
"""
# get version numbers from frozen.txt
with open('frozen.txt', 'r') as frozen:
    packages = [line.rstrip() for line in frozen.readlines()]

# search pyproject.toml for lines containing package
with open('../pyproject.toml', 'r') as project:
    lines = [line.rstrip() for line in project.readlines()]

# split line ->"    package",  ">=",  "version",
# replace version and rebuild line to update
for package in packages:
    name, version = package.split(">=")
    i, match = [(i, line) for i, line in enumerate(lines) if name in line][0]
    old_name, old_version = match.split(">=")
    new_line = ">=".join([old_name, version+'",'])
    lines[i] = new_line

# write out new pyproject.toml file
with open("../pyproject.toml", 'w') as output:
    output.write("\n".join(lines))


In [77]:
packages

['libpysal>=4.11.0',
 'access>=1.1.9',
 'esda>=2.5.1',
 'giddy>=2.3.5',
 'inequality>=1.0.1',
 'pointpats>=2.5.0',
 'segregation>=2.5',
 'spaghetti>=1.7.6',
 'mgwr>=2.2.1',
 'momepy>=0.7.2',
 'spglm>=1.1.0',
 'spint>=1.0.7',
 'spreg>=1.5.0',
 'spvcm>=0.3.0',
 'tobler>=0.11.2',
 'mapclassify>=2.7.0',
 'splot>=1.1.5.post1',
 'spopt>=0.6.1']